In [1]:
# https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html
print("pytorch neural networks!!!")


pytorch neural networks!!!


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        # an affine operation: y = Wx + b
        # 6*6还是5*5？
        self.fc1 = nn.Linear(16*6*6,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self, x):
        # max pooling over a (2,2) window
        print('x.size() before conv1:', x.size())
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        print('x.size() before conv2:', x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        print('x.size() before flat:', x.size())
        x = x.view(-1, self.num_flat_features(x))
        print('x.size() before fc1:', x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        # all dims except the bathc dim
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

        
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())



10
torch.Size([6, 1, 3, 3])


In [4]:
for i in range(len(params)):
    print(params[i].size())
    


torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [5]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)


x.size() before conv1: torch.Size([1, 1, 32, 32])
x.size() before conv2: torch.Size([1, 6, 15, 15])
x.size() before flat: torch.Size([1, 16, 6, 6])
x.size() before fc1: torch.Size([1, 576])
tensor([[ 0.0542, -0.0921,  0.0325,  0.0641, -0.1054,  0.0442, -0.0699, -0.0280,
          0.0352, -0.0377]], grad_fn=<AddmmBackward>)


In [6]:
net.zero_grad()
out.backward(torch.randn(1,10))


In [9]:
output = net(input)
target = torch.rand(10)
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)
print(criterion.type)



x.size() before conv1: torch.Size([1, 1, 32, 32])
x.size() before conv2: torch.Size([1, 6, 15, 15])
x.size() before flat: torch.Size([1, 16, 6, 6])
x.size() before fc1: torch.Size([1, 576])
tensor(0.2981, grad_fn=<MseLossBackward>)
<bound method Module.type of MSELoss()>


In [11]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])


None


In [9]:
# back prop
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0132, -0.0018, -0.0041,  0.0003, -0.0059, -0.0052])


In [10]:

import torch.optim as optim

optimizer =  optim.SGD(net.parameters(), lr=0.01)

# in your training loop.
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()



In [13]:
# max pool 2d
print('max pool 2d.')
mp = nn.MaxPool2d(2,stride=1)
input_mp = torch.randn((16,10,10))
output_mp = mp(input_mp)
print(output_mp.size())


max pool 2d.
torch.Size([16, 9, 9])
